In [82]:
import torch
import numpy as np

In [83]:
# Inputs
inputs = np.array([[70,40,60],[100,60,90,],[90,50,130],[80,30,40],[60,70,100]], dtype = 'float32')

In [84]:
#targets
targets = np.array([[58,77],[78,100],[122,130],[25,35],[113,120]],dtype='float32')

In [85]:
# Tensor conversion of inputs and targets
input_tensor = torch.from_numpy(inputs)
target_tensor = torch.from_numpy(targets)
print(input_tensor)
print(target_tensor)

tensor([[ 70.,  40.,  60.],
        [100.,  60.,  90.],
        [ 90.,  50., 130.],
        [ 80.,  30.,  40.],
        [ 60.,  70., 100.]])
tensor([[ 58.,  77.],
        [ 78., 100.],
        [122., 130.],
        [ 25.,  35.],
        [113., 120.]])


In [86]:
#Initializing random weights and biases
weights = torch.randn(2,3, requires_grad = True) #randn will pick random elements with mean = 0 and std=1
biases = torch.randn(2, requires_grad = True)

In [87]:
print(weights)
print(biases)

tensor([[-0.0779, -0.1709,  1.4723],
        [ 0.4338, -0.2397,  0.3794]], requires_grad=True)
tensor([ 0.4299, -1.7116], requires_grad=True)


In [88]:
input_tensor @ weights.t() + biases # '@' is for pytorch matrix multipication
# 't' is transpose of the tensor

tensor([[ 76.4757,  41.8254],
        [114.8883,  61.4252],
        [176.2698,  74.6598],
        [ 47.9594,  40.9728],
        [131.0197,  45.4711]], grad_fn=<AddBackward0>)

In [89]:
#Linear regression
def linear_regression_model(x):
    return x @ weights.t() + biases

In [90]:
#predictions from the model function
prediction = linear_regression_model(input_tensor)
print(prediction)

tensor([[ 76.4757,  41.8254],
        [114.8883,  61.4252],
        [176.2698,  74.6598],
        [ 47.9594,  40.9728],
        [131.0197,  45.4711]], grad_fn=<AddBackward0>)


In [91]:
#compare with targets
print(target_tensor)

tensor([[ 58.,  77.],
        [ 78., 100.],
        [122., 130.],
        [ 25.,  35.],
        [113., 120.]])


In [92]:
# Loss Function
difference = prediction - target_tensor
print(difference)

tensor([[ 18.4757, -35.1746],
        [ 36.8883, -38.5748],
        [ 54.2698, -55.3402],
        [ 22.9594,   5.9728],
        [ 18.0197, -74.5289]], grad_fn=<SubBackward0>)


In [93]:
difference * difference

tensor([[ 341.3521, 1237.2491],
        [1360.7488, 1488.0175],
        [2945.2068, 3062.5405],
        [ 527.1320,   35.6745],
        [ 324.7096, 5554.5522]], grad_fn=<MulBackward0>)

In [94]:
# take an average
torch.sum(difference * difference)/10

tensor(1687.7184, grad_fn=<DivBackward0>)

In [95]:
# we can get the number of elements using
torch.sum(difference * difference)/ difference.numel()

tensor(1687.7184, grad_fn=<DivBackward0>)

In [96]:
# Compute the Mean Squared Error for the model
def meanSquaredError(predictionsTensor , targetsTensor):
    difference = predictionsTensor - targetsTensor
    return torch.sum(difference * difference)/ difference.numel()

In [97]:
# Compute the information (f the loss is low, better the model)
model_loss = meanSquaredError(predictionsTensor = prediction, targetsTensor=target_tensor)
print(model_loss)


tensor(1687.7184, grad_fn=<DivBackward0>)


In [98]:
model_loss.backward()

In [99]:
print('Matrix of the weights')
print(weights)

Matrix of the weights
tensor([[-0.0779, -0.1709,  1.4723],
        [ 0.4338, -0.2397,  0.3794]], requires_grad=True)


In [100]:
print(weights.grad)

tensor([[ 2556.8684,  1523.1952,  2840.7812],
        [-3058.8459, -2305.2639, -3998.0825]])


In [101]:
print(biases)
print(biases.grad)

tensor([ 0.4299, -1.7116], requires_grad=True)
tensor([ 30.1226, -39.5291])


In [102]:
 with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5 # 1e-5 means the learning rate which determines how large your steps are
        

In [103]:
print(weights)

tensor([[-0.1035, -0.1862,  1.4439],
        [ 0.4644, -0.2167,  0.4194]], requires_grad=True)


In [104]:
print('Previous Predictions')
print(prediction)
print('Previous loss')
print(model_loss)

##new prediction
new_preds = linear_regression_model(input_tensor)
new_loss = meanSquaredError(predictionsTensor=new_preds, targetsTensor=target_tensor)
print('new Predictions')
print(new_preds)
print('new loss')
print(new_loss)

Previous Predictions
tensor([[ 76.4757,  41.8254],
        [114.8883,  61.4252],
        [176.2698,  74.6598],
        [ 47.9594,  40.9728],
        [131.0197,  45.4711]], grad_fn=<AddBackward0>)
Previous loss
tensor(1687.7184, grad_fn=<DivBackward0>)
new Predictions
tensor([[ 72.3719,  47.2880],
        [108.8605,  69.4659],
        [169.5136,  83.7633],
        [ 44.3203,  45.7111],
        [125.5783,  52.9186]], grad_fn=<AddBackward0>)
new loss
tensor(1251.5569, grad_fn=<DivBackward0>)


In [116]:
weights.grad.zero_()
biases.grad.zero_()

tensor([0., 0.])

In [119]:
for i in range(2000):
    print('Epochs number ' + str(i))
    
    # Forward pass
    predictions = linear_regression_model(input_tensor)
    
    # Compute loss
    loss = torch.nn.functional.mse_loss(predictions, target_tensor)
    print("Loss " + str(loss.item()))  # Convert loss tensor to scalar using .item()
    
    # Backward pass
    loss.backward()
    
    # Update weights and biases using gradient descent
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5 
        
        # Manually zero the gradients after updating weights and biases
        weights.grad.zero_()
        biases.grad.zero_()

Epochs number 0
Loss 1251.556884765625
Epochs number 1
Loss 948.6842651367188
Epochs number 2
Loss 738.2310180664062
Epochs number 3
Loss 591.82080078125
Epochs number 4
Loss 489.80218505859375
Epochs number 5
Loss 418.5564880371094
Epochs number 6
Loss 368.6432800292969
Epochs number 7
Loss 333.5203857421875
Epochs number 8
Loss 308.6528625488281
Epochs number 9
Loss 290.8978271484375
Epochs number 10
Loss 278.07696533203125
Epochs number 11
Loss 268.6802978515625
Epochs number 12
Loss 261.6612243652344
Epochs number 13
Loss 256.29461669921875
Epochs number 14
Loss 252.0774688720703
Epochs number 15
Loss 248.6613311767578
Epochs number 16
Loss 245.8046112060547
Epochs number 17
Loss 243.33999633789062
Epochs number 18
Loss 241.1509552001953
Epochs number 19
Loss 239.1573486328125
Epochs number 20
Loss 237.30322265625
Epochs number 21
Loss 235.5498046875
Epochs number 22
Loss 233.8701629638672
Epochs number 23
Loss 232.2456817626953
Epochs number 24
Loss 230.6634063720703
Epochs number

In [118]:
print('Predictions')
print(predictions)

print('Targets')
print(target_tensor)

Predictions
tensor([[ 72.3719,  47.2880],
        [108.8605,  69.4659],
        [169.5136,  83.7633],
        [ 44.3203,  45.7111],
        [125.5783,  52.9186]], grad_fn=<AddBackward0>)
Targets
tensor([[ 58.,  77.],
        [ 78., 100.],
        [122., 130.],
        [ 25.,  35.],
        [113., 120.]])
